# **Importing Necessary Libraries**

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# **Loading the Models and the Data**

In [4]:
x = np.load('data/x_test.npy')
y = np.load('data/y_test.npy')

In [5]:
gru_lstm = load_model('models/gru_lstm.h5', compile=False)

In [6]:
tdnn = load_model('models/tdnn_v2.h5', compile=False)

In [7]:
transformer = load_model('models/transformer.keras', compile=False)

In [8]:
x_train = x[:int(len(x)*0.5), :, :]
y_train = y[:int(len(y)*0.5), :]

x_test = x[int(len(x)*0.5):, :, :]
y_test = y[int(len(y)*0.5):, :]

In [9]:
gru_lstm_preds = gru_lstm.predict(x_train)
gru_lstm_preds_test = gru_lstm.predict(x_test)

1508/1508 ━━━━━━━━━━━━━━━━━━━━ 70s 45ms/step
1508/1508 ━━━━━━━━━━━━━━━━━━━━ 51s 34ms/step


In [10]:
tdnn_preds = tdnn.predict(x_train)
tdnn_preds_test = tdnn.predict(x_test)

1508/1508 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1508/1508 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step


In [11]:
transformer_preds = transformer.predict(x_train)
transformer_preds_test = transformer.predict(x_test)

1508/1508 ━━━━━━━━━━━━━━━━━━━━ 38s 25ms/step
1508/1508 ━━━━━━━━━━━━━━━━━━━━ 38s 25ms/step


# **Calculating the Weighted Score**

In [18]:
def weighted_score(y, y_pred, metrics):
    y_true15 = y[:, 0]
    y_true25 = y[:, 1]
    y_true30 = y[:, 2]
    
    y_pred15 = y_pred[:, 0]
    y_pred25 = y_pred[:, 1]
    y_pred30 = y_pred[:, 2]
    
    res = []
    
    for metric in metrics:
        score15 = metric(y_true15, y_pred15)
        score25 = metric(y_true25, y_pred25)    
        score30 = metric(y_true30, y_pred30)
    
        res.append((0.5*score15) + (0.35*score25) + (0.15*score30))
        
    return res

In [19]:
metrics = [r2_score, mean_absolute_error, mean_squared_error]

In [17]:
r2score, mae, mse = weighted_score(y_train, gru_lstm_preds, metrics)

print()
print(f'R2 Score: {r2score}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')


R2 Score: 0.8812509374845375
MAE: 5.807833765071466
MSE: 157.87215729289665


In [99]:
r2score, mae, mse = weighted_score(y_train, tdnn_preds, metrics)

print()
print(f'R2 Score: {r2score}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')


R2 Score: 0.8755107568270452
MAE: 5.579309439105898
MSE: 165.5034984128289


In [100]:
r2score, mae, mse = weighted_score(y_train, transformer_preds, metrics)

print()
print(f'R2 Score: {r2score}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')


R2 Score: 0.8717711009556905
MAE: 5.562907132974375
MSE: 170.47520470973427


# **Creating a Mean Ensemble**

In [18]:
ensemble_preds = (tdnn_preds + gru_lstm_preds + transformer_preds) / 3

In [19]:
r2score, mae, mse = weighted_score(y_train, ensemble_preds, metrics)

print()
print(f'R2 Score: {r2score}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')


R2 Score: 0.8895988639730954
MAE: 5.3363345210938675
MSE: 146.77387684737087


# **Creating a Meta Learner**

In [12]:
meta_x_train = np.hstack((gru_lstm_preds, transformer_preds, tdnn_preds))
meta_y_train = y_train

In [13]:
meta_x_test = np.hstack((gru_lstm_preds_test, transformer_preds_test, tdnn_preds_test))
meta_y_test = y_test

In [27]:
meta_learner = XGBRegressor(n_jobs=-1)
meta_learner.fit(meta_x_test, meta_y_test)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=-1,
             num_parallel_tree=None, random_state=None, ...)

In [28]:
preds = meta_learner.predict(meta_x_train)

In [ ]:
r2score, mae, mse = weighted_score(meta_y_train, preds, metrics)

print()
print(f'R2 Score: {r2score}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')


R2 Score: 0.9605137696190778
MAE: 3.166238729863045
MSE: 52.495295353533365
